# Laboratorio 9 - Algoritmo Forward Backward en HMM

#### Michelle Mejía y Silvia Illescas



------------------


### Task 1


1. Diga cual es la diferencia entre Modelos de Markov y Hidden Markov Models

Modelo de Markov: Es un modelo estadístico que predice el estado futuro de un sistema basado únicamente en el estado actual, sin tener en cuenta los estados anteriores. Es decir, el futuro depende solo del presente y no del pasado (propiedad de "memoria corta"). Se utiliza en sistemas donde los estados son observables.

Modelo de Markov Oculto (HMM): Es una extensión del modelo de Markov donde los estados no son directamente observables. En lugar de eso, el modelo genera observaciones que están relacionadas con los estados ocultos. En un HMM, hay una secuencia de estados ocultos que se pueden inferir a partir de observaciones visibles, y el modelo asume que la transición entre estos estados ocultos sigue las reglas de un proceso de Markov.

2. Investigue qué son los factorial HMM (Hidden Markov Models)

Un Factorial Hidden Markov Model (FHMM) es una extensión del modelo de Markov oculto (HMM) donde se modelan múltiples cadenas de Markov simultáneamente. Cada cadena representa un proceso de Markov independiente, pero todos ellos comparten las mismas observaciones, lo que permite modelar interacciones complejas entre diferentes procesos. Es especialmente útil en problemas donde se requiere modelar múltiples fuentes de variabilidad o donde los estados observados dependen de varios factores.

3. Especifique en sus propias palabras el algoritmo Forward Backward para HMM

El algoritmo Forward-Backward es un algoritmo utilizado en los Modelos de Markov Ocultos (HMM) para calcular la probabilidad de las secuencias observadas y para inferir la secuencia de estados ocultos más probable.

Paso Forward: En este paso, se calcula la probabilidad de llegar a cada estado oculto en el tiempo t dado las observaciones hasta ese momento. Se utiliza la probabilidad acumulada de las observaciones anteriores para calcular la probabilidad de los estados actuales.

Paso Backward: En este paso, se calcula la probabilidad de las observaciones futuras dado un estado oculto en el tiempo t. Esto se hace trabajando desde el último paso hacia atrás, utilizando las probabilidades de las observaciones futuras para actualizar la probabilidad de estar en un estado oculto en cualquier punto dado.

Este algoritmo se usa comúnmente en tareas como la decodificación y en la estimación de parámetros.

4. En el algoritmo de Forward Backward, por qué es necesario el paso de Backward (puede escribir ejemplos
o casos para responder esta pregunta)

El paso Backward es necesario porque, aunque el algoritmo Forward permite calcular la probabilidad de las observaciones pasadas, solo el paso Backward puede capturar la información sobre las observaciones futuras. Al combinar ambos pasos, se obtiene una estimación más precisa de las probabilidades de estar en un determinado estado en un momento dado.

Ejemplo de por qué es necesario el paso de Backward: Siguiendo el Task2, al estar modelando el clima con dos estados posibles: soleado y lluvioso. Si tenemos una secuencia de observaciones como "soleado, lluvioso, lluvioso", el algoritmo Forward solo podría calcular la probabilidad de que la primera observación haya sido soleada, pero el paso Backward permite ajustar esa probabilidad al considerar las observaciones futuras, es decir, que después de un "soleado" es más probable que venga un "lluvioso". Sin este paso, no obtendríamos una visión completa del sistema, ya que no se considerarían todas las observaciones en la secuencia.

--------------

### Task 2


In [2]:
class HMM:
    def __init__(self, states, observations, initial_prob, transition_prob, emission_prob):
        # Inicializar parámetros de HMM
        self.states = states
        self.observations = observations
        self.initial_prob = initial_prob
        self.transition_prob = transition_prob
        self.emission_prob = emission_prob

    def generate_sequence(self, length):
        # Generar una secuencia de observaciones basada en el HMM
        sequence = []
        state = np.random.choice(self.states, p=list(self.initial_prob.values()))
        for _ in range(length):
            sequence.append(np.random.choice(self.observations, p=list(self.emission_prob[state].values())))
            state = np.random.choice(self.states, p=list(self.transition_prob[state].values()))
        return sequence

    def forward(self, observations):
        # Implementar el paso hacia adelante del algoritmo hacia atrás-adelante
        alpha = [{}]
        for state in self.states:
            alpha[0][state] = self.initial_prob[state] * self.emission_prob[state][observations[0]]

        for t in range(1, len(observations)):
            alpha.append({})
            for curr_state in self.states:
                alpha[t][curr_state] = sum(
                    alpha[t - 1][prev_state] * self.transition_prob[prev_state][curr_state] for prev_state in self.states
                ) * self.emission_prob[curr_state][observations[t]]
        return alpha

    def backward(self, observations):
        # Implementar el paso hacia atrás del algoritmo hacia atrás-adelante
        beta = [{} for _ in range(len(observations))]
        for state in self.states:
            beta[-1][state] = 1  # paso base

        for t in reversed(range(len(observations) - 1)):
            for curr_state in self.states:
                beta[t][curr_state] = sum(
                    self.transition_prob[curr_state][next_state] *
                    self.emission_prob[next_state][observations[t + 1]] *
                    beta[t + 1][next_state]
                    for next_state in self.states
                )
        return beta

    def compute_state_probabilities(self, observations):
        # Combina probabilidades hacia adelante y hacia atrás para calcular probabilidades de estado
        alpha = self.forward(observations)
        beta = self.backward(observations)
        probs = []

        for t in range(len(observations)):
            total = sum(alpha[t][s] * beta[t][s] for s in self.states)
            probs.append({s: (alpha[t][s] * beta[t][s]) / total for s in self.states})

        return probs

# Uso y datos
states = ['Sunny', 'Rainy']
observations = ['sunny', 'rainy']
initial_prob = {'Sunny': 0.6, 'Rainy': 0.4}
transition_prob = {'Sunny': {'Sunny': 0.8, 'Rainy': 0.2}, 'Rainy': {'Sunny': 0.4, 'Rainy': 0.6}}
emission_prob = {'Sunny': {'sunny': 0.9, 'rainy': 0.1}, 'Rainy': {'sunny': 0.2, 'rainy': 0.8}}

hmm = HMM(states, observations, initial_prob, transition_prob, emission_prob)

# Generar una secuencia de observaciones.
obs_sequence = hmm.generate_sequence(5)
print("Secuencia Generada:", obs_sequence)

# Cálculo de probabilidades forward
forward_probs = hmm.forward(obs_sequence)
print("\nProbabilidades Forward:")
print(forward_probs)

# Cálculo de probabilidades backward
backward_probs = hmm.backward(obs_sequence)
print("\nProbabilidades Backward:")
print(backward_probs)

# Calcular probabilidades de estado
state_probs = hmm.compute_state_probabilities(obs_sequence)
print("\nProbabilidades de Estados:")
print(state_probs)


Secuencia Generada: ['sunny', 'rainy', 'sunny', 'sunny', 'sunny']

Probabilidades Forward:
[{'Sunny': 0.54, 'Rainy': 0.08000000000000002}, {'Sunny': 0.04640000000000001, 'Rainy': 0.12480000000000002}, {'Sunny': 0.07833600000000002, 'Rainy': 0.016832000000000003}, {'Sunny': 0.06246144000000001, 'Rainy': 0.0051532800000000005}, {'Sunny': 0.04682741760000001, 'Rainy': 0.003116851200000001}]

Probabilidades Backward:
[{'Sunny': 0.07266816000000004, 'Rainy': 0.13379328000000001}, {'Sunny': 0.4210560000000001, 'Rainy': 0.24364800000000006}, {'Sunny': 0.5664000000000001, 'Rainy': 0.33120000000000005}, {'Sunny': 0.7600000000000001, 'Rainy': 0.48000000000000004}, {'Sunny': 1, 'Rainy': 1}]

Probabilidades de Estados:
[{'Sunny': 0.7856918790249664, 'Rainy': 0.21430812097503366}, {'Sunny': 0.3911759821379145, 'Rainy': 0.6088240178620855}, {'Sunny': 0.8883804181351834, 'Rainy': 0.11161958186481648}, {'Sunny': 0.9504733083608585, 'Rainy': 0.04952669163914159}, {'Sunny': 0.9375934161238536, 'Rainy': 

El código implementa un modelo de Markov oculto (HMM) que genera secuencias de observaciones basadas en probabilidades iniciales, de transición y de emisión definidas. Se utiliza para modelar sistemas en los que los estados no son observables directamente, pero influyen en las observaciones que se reciben. En este caso, el modelo tiene dos estados posibles, "Sunny" (soleado) y "Rainy" (lluvioso), con probabilidades de transición y emisión específicas para cada uno. El algoritmo Forward calcula las probabilidades acumuladas de los estados a medida que se observan los eventos, mientras que el algoritmo Backward calcula las probabilidades de observar el futuro desde cada estado en un tiempo dado. Al combinar estos dos pasos, el algoritmo Forward-Backward obtiene las probabilidades de los estados en cada momento, teniendo en cuenta tanto el pasado como el futuro. Los resultados muestran que en el primer paso, la probabilidad de que el clima sea soleado (Sunny) es significativamente mayor que el de lluvia (Rainy), y estas probabilidades cambian conforme avanza la secuencia de observaciones. Este análisis es útil para tareas como la estimación de estados ocultos y la predicción de secuencias futuras en sistemas probabilísticos.